## Feature Selection => Forward Selection

### Data Ingestion

In [43]:
from warnings import filterwarnings
filterwarnings('ignore')

In [44]:
import pandas as pd

In [45]:
path = r"C:\Users\admin\Documents\Sindhura\LMS Basic Python and DA\datasets\Cars93.csv"

In [46]:
df = pd.read_csv(path,na_values=["","NA"],keep_default_na=False)
df.head()

id Manufacturer    Model     Type  Min.Price  Price  Max.Price  MPG.city  \
0   1        Acura  Integra    Small       12.9   15.9       18.8        25   
1   2        Acura   Legend  Midsize       29.2   33.9       38.7        18   
2   3         Audi       90  Compact       25.9   29.1       32.3        20   
3   4         Audi      100  Midsize       30.8   37.7       44.6        19   
4   5          BMW     535i  Midsize       23.7   30.0       36.2        22   

   MPG.highway             AirBags  ... Passengers Length  Wheelbase  Width  \
0           31                None  ...          5    177        102     68   
1           25  Driver & Passenger  ...          5    195        115     71   
2           26         Driver only  ...          5    180        102     67   
3           26                 NaN  ...          6    193        106     70   
4           30         Driver only  ...          4    186        109     69   

   Turn.circle  Rear.seat.room Luggage.room  Weight   Origin           Make  
0           37            26.5         11.0    2705  non-USA  Acura Integra  
1           38            30.0         15.0    3560  non-USA   Acura Legend  
2           37            28.0         14.0    3375  non-USA        Audi 90  
3           37            31.0         17.0    3405  non-USA       Audi 100  
4           39            27.0         13.0    3640  non-USA       BMW 535i  

[5 rows x 28 columns]

## Perform basic data quality checks

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  93 non-null     int64  
 1   Manufacturer        93 non-null     object 
 2   Model               93 non-null     object 
 3   Type                93 non-null     object 
 4   Min.Price           93 non-null     float64
 5   Price               93 non-null     float64
 6   Max.Price           93 non-null     float64
 7   MPG.city            93 non-null     int64  
 8   MPG.highway         93 non-null     int64  
 9   AirBags             89 non-null     object 
 10  DriveTrain          93 non-null     object 
 11  Cylinders           93 non-null     object 
 12  EngineSize          93 non-null     float64
 13  Horsepower          93 non-null     int64  
 14  RPM                 93 non-null     int64  
 15  Rev.per.mile        93 non-null     int64  
 16  Man.trans.

In [48]:
m = df.isna().sum()
m[m>0]

AirBags            4
Rear.seat.room     2
Luggage.room      11
dtype: int64

In [49]:
df.duplicated().sum()

0

## Drop insignificant columns

In [50]:
df.drop(columns="id",inplace=True)

## Separate X and Y

In [51]:
X = df.drop(columns="Weight")
Y = df[["Weight"]]

In [52]:
X.head()

Manufacturer    Model     Type  Min.Price  Price  Max.Price  MPG.city  \
0        Acura  Integra    Small       12.9   15.9       18.8        25   
1        Acura   Legend  Midsize       29.2   33.9       38.7        18   
2         Audi       90  Compact       25.9   29.1       32.3        20   
3         Audi      100  Midsize       30.8   37.7       44.6        19   
4          BMW     535i  Midsize       23.7   30.0       36.2        22   

   MPG.highway             AirBags DriveTrain  ... Fuel.tank.capacity  \
0           31                None      Front  ...               13.2   
1           25  Driver & Passenger      Front  ...               18.0   
2           26         Driver only      Front  ...               16.9   
3           26                 NaN      Front  ...               21.1   
4           30         Driver only       Rear  ...               21.1   

   Passengers  Length  Wheelbase  Width Turn.circle  Rear.seat.room  \
0           5     177        102     68          37            26.5   
1           5     195        115     71          38            30.0   
2           5     180        102     67          37            28.0   
3           6     193        106     70          37            31.0   
4           4     186        109     69          39            27.0   

   Luggage.room   Origin           Make  
0          11.0  non-USA  Acura Integra  
1          15.0  non-USA   Acura Legend  
2          14.0  non-USA        Audi 90  
3          17.0  non-USA       Audi 100  
4          13.0  non-USA       BMW 535i  

[5 rows x 26 columns]

In [53]:
Y.head()

Weight
0    2705
1    3560
2    3375
3    3405
4    3640

## Data PreProcessing and Data Cleaning

In [54]:
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

In [55]:
cat = list(X.columns[X.dtypes=="object"])
con = list(X.columns[X.dtypes!="object"])

In [56]:
cat_pipe = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OrdinalEncoder()
)

In [57]:
con_pipe = make_pipeline(
    SimpleImputer(strategy="mean"),
    StandardScaler()
)

In [58]:
pre = ColumnTransformer([("cat",cat_pipe,cat)
                         ,("con",con_pipe,con)]).set_output(transform="pandas")

In [59]:
pre

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder())]),
                                 ['Manufacturer', 'Model', 'Type', 'AirBags',
                                  'DriveTrain', 'Cylinders', 'Man.trans.avail',
                                  'Origin', 'Make']),
                                ('con',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Min.Price', 'Price', 'Max.Price', 'MPG.city',
                                  'MPG.highway', 'EngineSize', 'Horsepower',
                                  'RPM', 'Rev.per.mile', 'Fuel.tank.capacity',
                                  'Passengers', 'Length', 'Wheelbase', 'Width',
                                  'Turn.circle', 'Rear.seat.room',
                                  'Luggage.room'])])

In [60]:
X_pre = pre.fit_transform(X)

In [61]:
X_pre

cat__Manufacturer  cat__Model  cat__Type  cat__AirBags  cat__DriveTrain  \
0                 0.0        48.0        3.0           2.0              1.0   
1                 0.0        55.0        2.0           0.0              1.0   
2                 1.0         8.0        0.0           1.0              1.0   
3                 1.0         0.0        2.0           1.0              1.0   
4                 2.0         5.0        2.0           1.0              2.0   
..                ...         ...        ...           ...              ...   
88               30.0        41.0        5.0           2.0              1.0   
89               30.0        64.0        0.0           2.0              1.0   
90               30.0        28.0        4.0           2.0              1.0   
91               31.0         2.0        0.0           1.0              2.0   
92               31.0         7.0        2.0           0.0              1.0   

    cat__Cylinders  cat__Man.trans.avail  cat__Origin  cat__Make  \
0              1.0                   1.0          1.0        0.0   
1              3.0                   1.0          1.0        1.0   
2              3.0                   1.0          1.0        3.0   
3              3.0                   1.0          1.0        2.0   
4              1.0                   1.0          1.0        4.0   
..             ...                   ...          ...        ...   
88             2.0                   1.0          1.0       88.0   
89             1.0                   1.0          1.0       90.0   
90             3.0                   1.0          1.0       87.0   
91             1.0                   1.0          1.0       91.0   
92             2.0                   1.0          1.0       92.0   

    con__Min.Price  ...  con__RPM  con__Rev.per.mile  con__Fuel.tank.capacity  \
0        -0.485787  ...  1.717489           1.129530                -1.062184   
1         1.388017  ...  0.369586           0.005661                 0.409445   
2         1.008658  ...  0.369586          -0.105713                 0.072197   
3         1.571949  ...  0.369586           0.410659                 1.359872   
4         0.755752  ...  0.706562           0.430909                 1.359872   
..             ...  ...       ...                ...                      ...   
88       -0.060445  ... -1.315292           1.180155                 1.359872   
89        0.054512  ...  0.875050           0.714407                 0.562740   
90        0.663786  ...  0.875050           0.106911                 0.562740   
91        0.537333  ...  0.201098          -0.237337                -0.265051   
92        0.882205  ...  1.549001          -0.044963                 0.808011   

    con__Passengers  con__Length  con__Wheelbase  con__Width  \
0         -0.083243    -0.427186       -0.286932   -0.366184   
1         -0.083243     0.812171        1.629649    0.431983   
2         -0.083243    -0.220626       -0.286932   -0.632239   
3          0.884457     0.674465        0.302785    0.165927   
4         -1.050944     0.192493        0.745073   -0.100128   
..              ...          ...             ...         ...   
88         1.852158     0.261346        1.629649    0.698038   
89        -0.083243    -0.220626       -0.139503   -0.632239   
90        -1.050944    -1.666543       -1.024079   -0.898295   
91        -0.083243     0.467905        0.007926   -0.632239   
92        -0.083243     0.054786        0.155356   -0.100128   

    con__Turn.circle  con__Rear.seat.room  con__Luggage.room  
0          -0.610436            -0.452197          -1.033015  
1          -0.298510             0.738090           0.396643  
2          -0.610436             0.057926           0.039228  
3          -0.610436             1.078172           1.111472  
4           0.013416            -0.282156          -0.318186  
..               ...                  ...                ...  
88         -0.298510             2.098419           0.0

## Feature Selection => Forward Selection

In [62]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression

    base_model = LinearRegression()
    for_sel= SequentialFeatureSelector(base_model,direction="forward",n_features_to_select=10)
    for_sel.fit(X_pre,Y)
    for_sel.get_feature_names_out

In [63]:
base_model = LinearRegression()
for_sel= SequentialFeatureSelector(base_model,direction="forward",n_features_to_select=10)

In [64]:
for_sel.fit(X_pre,Y)

SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select=10)

In [65]:
imp_cols = for_sel.get_feature_names_out()
imp_cols

array(['cat__Manufacturer', 'cat__Origin', 'con__MPG.highway',
       'con__Horsepower', 'con__RPM', 'con__Fuel.tank.capacity',
       'con__Passengers', 'con__Wheelbase', 'con__Width',
       'con__Rear.seat.room'], dtype=object)

In [66]:
len(imp_cols)

10

In [67]:
imp_cols

array(['cat__Manufacturer', 'cat__Origin', 'con__MPG.highway',
       'con__Horsepower', 'con__RPM', 'con__Fuel.tank.capacity',
       'con__Passengers', 'con__Wheelbase', 'con__Width',
       'con__Rear.seat.room'], dtype=object)

In [68]:
imp_cols[0]

'cat__Manufacturer'

In [69]:
imp_cols[0].split("__")

['cat', 'Manufacturer']

In [70]:
imp_cols[0].split("__")[1]

'Manufacturer'

In [71]:
sel_cols = [col.split("__")[1] for col in imp_cols]
sel_cols

['Manufacturer',
 'Origin',
 'MPG.highway',
 'Horsepower',
 'RPM',
 'Fuel.tank.capacity',
 'Passengers',
 'Wheelbase',
 'Width',
 'Rear.seat.room']